In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib
import os
import cv2
import seaborn as sns
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Flatten
from tensorflow.keras import Input, Model
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available')
else:
    print('No GPU detected')

num_gpus = len(physical_devices)

if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {tf.config.experimental.get_device_details(physical_devices[0])}")
else:
    print("No GPUs available")

device = tf.device('gpu:0' if len(physical_devices) > 0 else 'cpu:0')


In [ ]:
train = r'D:\Brain Cancer\Dataset\Brain_Cancer_processed_train_test\train_augmented'
val = r'D:\Brain Cancer\Dataset\Brain_Cancer_processed_train_test\val'
test = r'D:\Brain Cancer\Dataset\Brain_Cancer_processed_train_test\test'

import os

print(os.listdir(train))
print(os.listdir(val))
print(os.listdir(test))

In [ ]:
tf.random.set_seed(42)

train_set = keras.utils.image_dataset_from_directory(train, seed = 1, shuffle = True, batch_size = 128, image_size=(128,128))

val_set = keras.utils.image_dataset_from_directory(val, seed = 1, shuffle = True, batch_size = 16, image_size=(128,128))

test_set = keras.utils.image_dataset_from_directory(test, seed = 1, shuffle = True, batch_size = 16, image_size=(128,128))

In [ ]:
filenames = pathlib.Path(train)
for label in train_set.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
filenames = pathlib.Path(val)
for label in val_set.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
filenames = pathlib.Path(test)
for label in test_set.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
train_set.cardinality().numpy(), val_set.cardinality().numpy(), test_set.cardinality().numpy()

In [ ]:
# # print random images from the train set
# plt.figure(figsize = (15, 15))
# for images, labels in train_set.take(1):
#     for i in range(15):
#         index = random.randint(0, len(images))
#         ax = plt.subplot(3, 5, i + 1)
#         plt.imshow(images[index].numpy().astype("uint8"))
#         plt.title(train_set.class_names[labels[index]], color= 'blue', fontsize= 12)
#         plt.axis(True)
# plt.show()

In [ ]:
for images_batch, labels_batch in train_set:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
MobileNetV2_2Model = MobileNetV2(include_top = False, weights = 'imagenet', input_shape = (128, 128, 3))

In [ ]:
MobileNetV2_2Model.summary()

In [ ]:
MobileNetV2_2Model.trainable = True

In [ ]:
tf.random.set_seed(42)

X = MobileNetV2_2Model.output

X = Flatten()(X)

X = Dropout(0.5)(X)
X = Dense(1024, activation='relu')(X)
X = Dropout(0.5)(X)
X = Dense(512, activation='relu')(X)
X = Dropout(0.25)(X)

output_layer = Dense(4, activation='softmax')(X)

model2 = Model(inputs = MobileNetV2_2Model.input, outputs = output_layer)

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history_2 = model2.fit(train_set, epochs=30, validation_data=val_set)

In [ ]:
# Save the model
#model2.save('d:\\Balanced Augmented Covid CXR Dataset\\Model\\MobileNetV2_2Model.h5')

In [ ]:
X_test, y_test = None, None
for images, labels in test_set:
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2) 
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
        # Specify the directory to save the PDF
    save_dir = 'D:\\Brain Cancer\\PDF'
    
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Full path to save the PDF file
    pdf_path = os.path.join(save_dir, 'MobileNetV2_2_training_curves.pdf')
    
    # Save the plot as a PDF with tight layout
    plt.tight_layout()  # Ensure that layout is tight
    plt.savefig(pdf_path, format='pdf', bbox_inches='tight')  
    # Show the plot
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history_2.history))

In [ ]:
y_pred_proba = model2.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
test_score = model2.evaluate(test_set, verbose= 1)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
target_names = ['brain_glioma', 'brain_menin', 'brain_tumor']
print(classification_report(y_test , y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
plt.figure(figsize = (6,6), dpi = 100)
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot = True, fmt='d', cmap = 'Greens')
plt.xlabel('Predictions')
plt.ylabel('True Labels')
    # Add a title with the model name MobileNetV2_2
plt.title('Confusion Matrix for MobileNetV2_2')

    # Specify the directory to save the PDF
save_dir = 'D:\\Brain Cancer\\PDF'
    
    # Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

    # Full path to save the PDF file (with MobileNetV2_2 in the filename)
pdf_path = os.path.join(save_dir, 'MobileNetV2_2_confusion_matrix.pdf')
    
    # Save the plot as a PDF with tight layout
plt.tight_layout()  # Ensure that layout is tight
plt.savefig(pdf_path, format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# plot random images from a given dataset, and compare predictions with ground truth
def plot_random_predictions(dataset, model):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        plt.figure(figsize = (10, 10), dpi = 120)
        y_pred_proba = model.predict(images)

    for i in range(9):
        index = random.randint(0, len(images))
        ax = plt.subplot(3,3, i + 1)

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]
      
        c = 'g' if y_pred == y_true else 'r'
      
        plt.imshow(img)
        plt.title(f'Predicted : {y_pred}\nTrue label : {y_true}', c = c)
        plt.axis(False)

In [ ]:
plot_random_predictions(test_set, model2)